This code segment builds upon the work of Joseph and others, primarily to validate the independent effect of metal. It was completed independently but utilizes prior results (which should be included after the initial PCA completion section). Unlike the other two sections, this part does not include an uploaded dataset and cannot be run locally.

In [ ]:
# ===================== KMeans clustering based on PCA =====================
# Requirements (already created in your notebook before this cell):
#   - pca_df with columns ['PC1', 'PC2', 'PC3']
#   - WQI_gdf_sorted
#   - HPI_gdf

import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from IPython.display import display
import matplotlib.pyplot as plt
import plotly.express as px

# 1. Prepare data for clustering: use the three principal components
X_cluster = pca_df[['PC1', 'PC2', 'PC3']].copy()

# 2. Try several values of k and compute inertia (SSE) and silhouette score
K_range = range(2, 8)
inertias = []
sil_scores = []

for k in K_range:
    km = KMeans(
        n_clusters=k,
        random_state=42,
        n_init=10
    )
    labels = km.fit_predict(X_cluster)
    inertias.append(km.inertia_)
    sil_scores.append(silhouette_score(X_cluster, labels))

# 3. Plot elbow curve and silhouette scores
fig, ax1 = plt.subplots(figsize=(8, 4))

ax1.plot(list(K_range), inertias, marker='o')
ax1.set_xlabel('Number of clusters (k)')
ax1.set_ylabel('Inertia (SSE)')
ax1.set_title('KMeans: Elbow and Silhouette')

ax2 = ax1.twinx()
ax2.plot(list(K_range), sil_scores, marker='s', linestyle='--')
ax2.set_ylabel('Silhouette score')

plt.tight_layout()
plt.show()

# 4. Choose best k by maximum silhouette score
best_k = list(K_range)[int(np.argmax(sil_scores))]
print(f"Best number of clusters by silhouette score: k = {best_k}")

# 5. Fit final KMeans model with best_k
best_kmeans = KMeans(
    n_clusters=best_k,
    random_state=42,
    n_init=10
)
cluster_labels = best_kmeans.fit_predict(X_cluster)

# Add cluster labels back to PCA dataframe
pca_df['kmeans_cluster'] = cluster_labels.astype(int)

# IMPORTANT FIX:
# Do NOT use .join() (it causes column overlap errors when re-running).
# Instead assign the column directly. This overwrites safely if it already exists.
WQI_gdf_sorted['kmeans_cluster'] = pca_df['kmeans_cluster']
HPI_gdf['kmeans_cluster'] = pca_df['kmeans_cluster']

# 6. Cluster-wise summary using WQI and HPI
cluster_analysis_df = pd.DataFrame({
    'WQI': WQI_gdf_sorted['WQI'],
    'HPI': HPI_gdf['HPI'],
    'cluster': pca_df['kmeans_cluster']
}).dropna(subset=['cluster'])

cluster_summary = (
    cluster_analysis_df
    .groupby('cluster')
    .agg(
        n_samples=('WQI', 'size'),
        WQI_mean=('WQI', 'mean'),
        WQI_std=('WQI', 'std'),
        HPI_mean=('HPI', 'mean'),
        HPI_std=('HPI', 'std')
    )
    .sort_index()
)

print("Cluster-wise summary for WQI and HPI:")
display(cluster_summary)

# 7. 3D scatter of PCA space colored by cluster
#    Use generic axis labels in case you don't have var_exp defined.
fig3d = px.scatter_3d(
    pca_df,
    x='PC1',
    y='PC2',
    z='PC3',
    color=pca_df['kmeans_cluster'].astype(str),
    hover_name=pca_df.index.astype(str),
    opacity=0.7
)

fig3d.update_layout(
    title=f"KMeans clustering in PCA space (k = {best_k})",
    width=900,
    height=700,
    scene=dict(
        xaxis_title="PC1",
        yaxis_title="PC2",
        zaxis_title="PC3",
    )
)
fig3d.show()

# 8. 2D scatter of WQI vs HPI colored by cluster
plt.figure(figsize=(7, 6))
for c in sorted(cluster_analysis_df['cluster'].unique()):
    sub = cluster_analysis_df[cluster_analysis_df['cluster'] == c]
    plt.scatter(sub['WQI'], sub['HPI'], label=f'Cluster {c}', alpha=0.6)

plt.xlabel('WQI')
plt.ylabel('HPI')
plt.title(f'KMeans clusters in WQI vs HPI space (k = {best_k})')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
import geopandas as gpd
import folium

# 1) One representative row per station (to get geometry)
station_locs = (
    WQI_gdf_sorted
    .sort_values('Collect_dt')
    .drop_duplicates(subset='Sampling')
    [['Sampling', 'geometry']]
    .set_index('Sampling')
)

# 2) For each station, take the most frequent cluster label
station_cluster = (
    WQI_gdf_sorted
    .groupby('Sampling')['kmeans_cluster']
    .agg(lambda x: x.value_counts().idxmax())
    .rename('kmeans_cluster')
)

# 3) Combine into a GeoDataFrame
station_cluster_gdf = gpd.GeoDataFrame(
    station_locs.join(station_cluster),
    geometry='geometry',
    crs=WQI_gdf_sorted.crs
)

# Optional: make a string label for legends / tooltips
station_cluster_gdf['cluster_label'] = station_cluster_gdf['kmeans_cluster'].apply(
    lambda c: f"Cluster {int(c)}"
)

station_cluster_gdf.head()


In [ ]:
sensitive_waterways = {
    "North Branch Canal": North_branch_canal,
    "North Shore Channel": North_shore_channel,
    "Chicago Sanitary & Ship Canal": Chicago_sanitary_ship_canal,
    "Little Calumet River": Little_calumet_river,
}


In [ ]:
# Focus on Chicago area streams
bbox = (-88.15, 41.55, -87.3, 42.10)
chicago_streams = streams.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]

# Base map with streams
m = chicago_streams.explore(
    tiles="CartoDB Positron",
    name="Streams",
    style_kwds={"color": "steelblue", "weight": 2}
)

# 3.1 All stations colored by KMeans cluster
station_cluster_gdf.explore(
    m=m,
    column='kmeans_cluster',          # color by cluster id
    categorical=True,
    cmap='Set1',
    legend=True,
    legend_kwds={'caption': 'KMeans cluster'},
    marker_kwds=dict(radius=6, fill=True),
    tooltip=['cluster_label', 'kmeans_cluster'],
    name='All stations by cluster',
)

# 3.2 Sensitive waterways highlighted (larger markers)
for waterway_name, site_list in sensitive_waterways.items():
    sub = station_cluster_gdf.loc[station_cluster_gdf.index.isin(site_list)].copy()
    if sub.empty:
        continue

    sub.explore(
        m=m,
        column='kmeans_cluster',
        categorical=True,
        cmap='Set1',
        legend=False,   # legend already added above
        marker_kwds=dict(radius=9, fill=True),
        tooltip=['cluster_label'],
        name=f'{waterway_name} (sensitive)',
    )

folium.LayerControl().add_to(m)
m
